<a href="https://colab.research.google.com/github/abi-kothapalli/MinimumDominatingSets/blob/main/ExactAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implements an exact algorithm to solve the minimum dominating set problem

Adapts the code from [this Github repo.](https://github.com/jward1/MinimumDominatingSet)

# Greate Graph classes


*   Edge
*   Graph
*   Nodes


In [24]:
public class Node
{
	private int name;
	private HashSet<Edge> nodesEdges;
	private int assignment;

	/**
	 * Creates a Node object for the Graph class.
	 * @param name The Integer name of the node.
	 */
	public Node(int name)
	{
		this.name = name;
		this.nodesEdges = new HashSet<Edge>();
		this.assignment = -1;
	}

	/**
	 * Returns the Integer name of the node.
	 * @return int The name of the node.
	 */
	public int getName() { return this.name; }


	/**
	 * Get the neighbors of a node.
	 * @return a HashSet of Nodes.
	 */
	public List<Node> getNeighbors() 
	{
		List<Node> neighbors = new ArrayList<Node>();
		for (Edge edge : nodesEdges) {
			neighbors.add(edge.getOtherNode(this));
		}
		return neighbors;
	}


	/**
	 * Get the number of a neighbors of a node.
	 * @return int the number of a node's neighbors
	 */
	public int numNeighbors() { return nodesEdges.size(); }


	/**
	 * Returns the Edges attached to the node.
	 * @return A HashSet of Edge objects connected to the Node.
	 */
	public HashSet<Edge> getEdges() { return this.nodesEdges; }
	

	/**
	 * Returns whether the node is assigned to the Dominating Set
	 * or Vertex Cover. { -1: Not assigned, 0: not in set, 1: assigned
	 * to set}
	 * @return int of the nodes assignement
	 */
	public int isAssigned() { return this.assignment; }


	/**
	 * Updates a node's assignment
	 * @param newAssignement The new assignement for a node. Takes -1, 0, or 1.
	 * @throws IllegalArgumentException
	 */
	public void setAssignment(int newAssignment) 
	{ 
		if (newAssignment == -1 || newAssignment == 0 || newAssignment == 1)
			this.assignment = newAssignment;
		else
			throw new IllegalArgumentException("assignment must be set to -1, 0, or 1.");
	}


	/**
	 * Adds an edge to the Node.
	 * @param newEdge the new Edge to be added to a Node.
	 */
	public void addEdge(Edge newEdge) 
	{ 
		if ( !nodesEdges.contains(newEdge) )
			nodesEdges.add(newEdge); 
	}


	/**
	 * Returns the name of the Node as a String.
	 * @return String of the Node's name.
	 */
	public String toString() { return Integer.toString(this.name); }
	
}

In [22]:
public class Edge
{
	private Node v1;
	private Node v2;
	private int isCovered; 

	/**
	 * Creates an Edge object between two nodes.
	 * @param node1 One end of the Edge.
	 * @param node2 The other end of the Edge.
	 */
	public Edge (Node node1, Node node2)
	{	
		// make sure lower value name is always stored in v1
		Node val1;
		Node val2;

		if      ( node1.getName() < node2.getName() )  { val1 = node1; val2 = node2; }
		else if ( node1.getName() > node2.getName() )  { val2 = node1; val1 = node2; }
		else { throw new IllegalArgumentException("Self loops are not allowed in this Graph."); }

		this.v1 = val1;
		this.v2 = val2;
		this.isCovered = -1;
	}


	/**
	 * Converts the edge object to a String
	 * @return A String representation of the Edge object.
	 */
	public String toString() 
	{ 	
		return v1.getName() + "---" + v2.getName();
	}
	

	/**
	 * Gets the node at the other end of the Edge.
	 * @param node One node contained in the edge
	 * @return The node at the other end of the edge
	 * @throws IllegalArgumentExepction if the node passed to the method
	 * 		   is not contained in the Edge.
	 */
	public Node getOtherNode(Node node)
	{
		if (node.equals(v1)) 
			return v2;
		else if (node.equals(v2))
			return v1;
		throw new IllegalArgumentException("Looking for " +
			"a point that is not in the edge");
	}


	/**
	 * Returns the v1 node.
	 * @return the v1 node of an edge.
	 */
	public Node getNodev1() { return this.v1; }


	/**
	 * Returns the v2 node.
	 * @return the v2 node of an edge.
	 */
	public Node getNodev2() { return this.v2; }


	/**
	 * Tests whether an Edge is equal to an Edge already in the graph.
	 * @param The other edge to be tested if equal to this Edge object.
	 * @return True if the Edge objects represent the same edge; otherwise False
	 */
	public boolean isEqualTo(Edge edge)
	{
		if ( this.v1.equals(edge.getNodev1()) && this.v2.equals(edge.getNodev2()) )
			return true;
		return false;
	}


	/**
	 * Returns the state of an Edge's coverage when looking for a
	 * Vertex Cover {-1: not assigned, 0: not covered, 1:covered}
	 * @return An integer representing an Edge's coverage.
	 */
	public int isCovered() { return this.isCovered; }


	/**
	 * Updates a node's coverage state
	 * @param newState The new state for an edge. Takes -1, 0, or 1.
	 * @throws IllegalArgumentException
	 */
	public void setIsCovered(int newState) 
	{ 
		if (newState == -1 || newState == 0 || newState == 1)
			this.isCovered = newState;
		else
			throw new IllegalArgumentException("isCovered must be set to -1, 0, or 1.");
	}

}

In [27]:
import java.io.File;
import java.util.Scanner;
import java.util.HashMap;
import java.util.HashSet;
import java.util.List;
import java.util.LinkedList;
import java.util.ArrayList;
import java.util.Collections;


public class Graph
{
	private HashMap<Integer, Node> nodes;	// The nodes of the Graph
	private HashSet<Edge> edges;			// The edges of the Graph; not sure if this is useful

	/** 
	 * Creates a new Graph object.
	 */
	public Graph()
	{
		nodes = new HashMap<Integer, Node>();
		edges = new HashSet<Edge>();
	}


	/** 
	 * Adds a node to the graph. Runs in O(1)
	 * @param key An integer that acts as node's name
	 */
	public void addNode(int key)
	{
		if ( nodes.get(key) == null)
		{
			Node newNode = new Node(key);
			nodes.put(key, newNode);
		}
		return;
	}


	/**
	 * Removes a node and all of its Edges from a Graph.
	 * @param nodeToRemove The Integer name of the node to be removed.
	 */
	public void removeNode(int nodeToRemove)
	{
		Node node = nodes.get(nodeToRemove);
		removeNodesEdges(node);
		nodes.put(nodeToRemove, null);
	}

	
	/**
	 * Removes a node and all of its Edges from a Graph.
	 * @param nodeToRemove The Node object to be removed.
	 */
	public void removeNode(Node nodeToRemove)
	{
		removeNodesEdges(nodeToRemove);
		nodes.put(nodeToRemove.getName(), null);
	}

	
	/**
	 * Removes the Edges from a graph for a given Node.
	 * @nodeToRemove The Integer name of the node to be removed.
	 */
	public void removeNodeEdges(int nodeToRemove)
	{
		Node node = nodes.get(nodeToRemove);
		removeNodesEdges(node);

	}


	/**
	 * Removes the Edges from a graph for a given Node.
	 * @param nodeToRemove The Node object to be removed.
	 */
	public void removeNodesEdges(Node nodeToRemove)
	{
		for (Edge edge : nodeToRemove.getEdges())
			edges.remove(edge);
	}

	
	/** 
	 * Adds an undirected edge to the graph.
	 * @param v1 The name of the Node at one end of the edge
	 * @param v2 The name of the Node at the other end of the edge
	 */
	public void addEdge(int v1, int v2)
	{	
		// ensure that the from and to nodes are in the Graph
		if (!nodes.containsKey(v1))	
            throw new NullPointerException("Node " + v1 + " is not contained in the Graph.");
		if (!nodes.containsKey(v2))
            throw new NullPointerException("Node " + v2 + " is not contained in the Graph.");
		
		// extract Node objects from Graph and construct new Edge object
		Node node1 = nodes.get(v1);
		Node node2 = nodes.get(v2);
		Edge newEdge = new Edge(node1, node2);

        // do not add new edge if already contained in graph
        for (Edge edge : node1.getEdges() )
        {
            if (edge.isEqualTo(newEdge)) { return; }
        }

		// if Edge is not in the Graph; add it
        edges.add(newEdge);
        node1.addEdge(newEdge);
        node2.addEdge(newEdge);
        
	}


	/**
	 * Adds an undirected edge to the graph. Useful for building 
	 * copies of a graph or separate subgraphs.
	 * @param newEdge The edge to be added to the graph.
	 */
	public void addEdge(Edge newEdge)
	{
		edges.add(newEdge);
		Node v1 = newEdge.getNodev1();
		Node v2 = newEdge.getNodev2();
		v1.addEdge(newEdge);
		v2.addEdge(newEdge);
	}


	/** 
	 * Returns the nodes of the Graph.
	 * @return A List of Nodes that contained in the Graph
	 */
	public List<Node> getNodes()
	{	return new ArrayList<Node>( nodes.values() );	}


    /** 
     * Returns the total number of nodes in the Graph.
     * @return An int representing the number of nodes in the Graph
     */
    public int getNumNodes()
    {   return nodes.size(); }


    /**
     *
     */
    public List<Edge> getEdges()
    {	return new ArrayList<Edge>(edges); }


	/** 
	 * Returns the total number of edges in the Graph.
	 * @return An int representing the number of edges in the Graph
	 */
    public int getNumEdges()
    {	return edges.size(); }


	/** 
	 * Returns the graph in String format. Runs in O(|V + E|).
	 * The line '0--> [ 1 2 ]' is read Node 0 is the starting point of edges connecting
	 * to Node 1 and Node 2
	 * @return A String representation of the Graph
	 */
	public String exportGraphString()
    {
    	String s = "";
        for (int key : nodes.keySet() )
        {
            s += "\n" + key + "--> [";
            Node node = nodes.get(key);
            
            for (Edge edge : node.getEdges())
            {
            	s += " " + (edge.getOtherNode(node)).toString();
            }
            s += " ]";
        }
        return s;
    }


    /**
     * Resets all Nodes and Edges in graph to uncovered.
     * Runs in O(|V| + |E|).
     */
    public void resetGraph()
    {
        for (Node node : nodes.values() )
            node.setAssignment(-1);

        for (Edge edge : edges)
        	edge.setIsCovered(-1);
        
        return;
    }
}

# Dominating Set Algorithm

Implements the aglorithm discussed in "Exact (exponential) algorithms for the dominating set problem", by Fomin, Kratsch, and Woeginger. Runs in $O(1.93782^n)$.

In [36]:
import java.util.List;
import java.util.ArrayList;

public class DominatingSet
{   
	/**
     * Returns a minimum dominating set of a graph.
     * Implements the aglorithm discussed in "Exact (exponential) algorithms for the 
     * dominating set problem", by Fomin, Kratsch, and Woeginger.
     * 
     * This algorithm runs in O(1.93782^n) where n is the number of nodes (or vertices)
     * in the graph.
     * 
     * @param graph The graph object for which a minimum dominating set is to be found.
     * @return A List of Nodes representing the nodes included in the dominating set.
     */
    public static List<Node> findMinimumDominatingSet(Graph graph)
    {   
        List<Node> mds = recursiveSearchTree(graph);
        return mds;
    }


    /**
     * This method implements the core of the algorithm developed by Fomin, et. al. 
     * 
     * The recursive search tree works finding all nodes with degree 1 and 2 (that is,
     * nodes with only 1 or 2 neighbors). 
     *      Case 1: If node v is of degree 1, then this algorithm assigns its neighbor, w,
     *      to be in the dominating set.
     *      Case 2: If the node v is of degree 2, then this algorithm splits into three
     *      subcases. Let the two neighbors be denoted as u1 and u2
     *          Case 2a: Mark u1 as in D, and v as not in D. Recurse on resulting graph.
     *          Case 2b: Mark v as in D, and u1 and u2 as not in D. Recurse on resulting graph.
     *          Case 2c: Mark u2 as in D, and u1 and v as not in D. Recurse on resulting graph.
     * After the search tree has dealt with all nodes of degree one and two, it calls the brute force
     * search to find the minimum dominating set on the "reduced" graph.
     * 
     * @param graph The graph object for which a minimum dominating set is to be found.
     * @return A List of Nodes representing the nodes included in the dominating set.
     */
    private static List<Node> recursiveSearchTree(Graph graph)
    {   
        // initialize node variables
        Node w1 = null;
        Node u1 = null;
        Node u2 = null;
        Node v  = null;

        // initialize results to all nodes in graph
        List<Node> wResult  = null;
        List<Node> u1Result = null;
        List<Node> u2Result = null;
        List<Node> vResult  = null;

        // set the size of the result sets to the max integer value
        int wSize  = Integer.MAX_VALUE;
        int u1Size = Integer.MAX_VALUE;
        int u2Size = Integer.MAX_VALUE;
        int vSize  = Integer.MAX_VALUE;

        // find nodes with either 1 or 2 uncovered neighbors
        for (Node n : graph.getNodes() )
        {
            if (n.isAssigned() == -1)
            {
                List<Node> unassigned = getNumUnassignedNeighbors(n);
                if (unassigned.size() == 1)
                {   
                    w1 = unassigned.get(0);
                    v = n;
                }
                else if (unassigned.size() == 2)
                {
                    u1 = unassigned.get(0);
                    u2 = unassigned.get(1);
                    v = n;
                }
            }
        }

        // if all are null, then graph only contains nodes with 
        // zero or three or more neighbors
        if (w1==null && u1==null && u2==null && v==null)
        {   
            int numNodesAssigned = 0;
            for (Node node : graph.getNodes())
                if (node.isAssigned() == 1)  { numNodesAssigned++; }

            return bruteForce(graph, graph.getNumNodes() - numNodesAssigned, numNodesAssigned);
        }
            

        //recursive branching

        // case 1
        if (w1 != null)
        {
            w1.setAssignment(1);
            v.setAssignment(0);
            wResult = recursiveSearchTree(graph);
            wSize = wResult.size();
            w1.setAssignment(-1);
            v.setAssignment(-1);
        }

        // case 2
        if (u1 != null)
        {   
            // case 2a
            u1.setAssignment(1);
            u2.setAssignment(-1);
            v.setAssignment(0);
            u1Result = recursiveSearchTree(graph);
            u1Size = u1Result.size();

            // case 2b
            u1.setAssignment(0);
            u2.setAssignment(0);
            v.setAssignment(1);
            vResult = recursiveSearchTree(graph);
            vSize = vResult.size();

            // case 2c
            u1.setAssignment(0);
            u2.setAssignment(1);
            v.setAssignment(0);
            u2Result = recursiveSearchTree(graph);
            u2Size = u2Result.size();

            // reset assignments
            u1.setAssignment(-1);
            u2.setAssignment(-1);
            v.setAssignment(-1);
        }

        // return smallest result
        int minNum = Math.min( wSize, Math.min(u1Size, Math.min(u2Size, vSize)));
        if      (minNum == wSize ) { return wResult;  }
        else if (minNum == u1Size) { return u1Result; }
        else if (minNum == u2Size) { return u2Result; }
        else                       { return vResult;  }
    }


    /**
     * Finds the minimum dominant set of a graph.
     *
     * This uses brute force, that is, it tries every possible combination of assignments,
     * to find a minimum dominant set of a graph. Given that there are 2 possible states for
     * each of the n nodes, this method runs in O(2^n).
     *
     * @param graph The graph object for which a minimum dominating set is to be found.
     * @return A List of Nodes representing the nodes included in the dominating set.
     */
    private static List<Node> bruteForce(Graph graph, int sizeOfSubgraph, int numAlreadyAssigned)
    {   
        
        Node u = null; 
        int totalNumAssignedToOne = 0;

        // check if dominating set is possible, if not return full graph as dominating set
        for (Node node : graph.getNodes() )
        {   
            if (node.isAssigned() == 0)
            {
                boolean allNeighborsSetToZero = true;
                
                for (Node neighbor : node.getNeighbors() ) 
                {   
                    if (neighbor.isAssigned() != 0)
                        allNeighborsSetToZero = false;
                }

                
                if (allNeighborsSetToZero)
                {
                    return (ArrayList<Node>) graph.getNodes();
                }  
            }
            else if (node.isAssigned() == 1)
                totalNumAssignedToOne++;
            else
                u = node;
        }

        // if all nodes have been assigned and set is valid, return dominating set
        if (u == null)
        {   
            List<Node> ds = new ArrayList<Node>();
            for (Node node : graph.getNodes())
            {
                if (node.isAssigned() == 1)
                    ds.add(node);
            }
            return ds;
        }

        // stop searching if number of assigned nodes is greater than or equal
        // to 3n/8
        if (totalNumAssignedToOne - numAlreadyAssigned >= 3*sizeOfSubgraph/8)
            return (ArrayList<Node>) graph.getNodes();

        // recursively iterate through all assignments of u
        u.setAssignment(0);
        List<Node> u0 = bruteForce(graph, sizeOfSubgraph, numAlreadyAssigned);
        u.setAssignment(1);
        List<Node> u1 = bruteForce(graph, sizeOfSubgraph, numAlreadyAssigned);
        u.setAssignment(-1);

        //return the smallest set
        if ( u0.size() < u1.size() )   { return u0; }
        else                           { return u1; }
    }


    /** 
     * Returns a list of unassigned neighbors of a given node.
     *
     * @param node The node for which to get the neighbors that have
     *                  have not been assigned to in or out of the dominating set.
     * @return A List of the unassigned neighbor nodes
     */
    private static List<Node> getNumUnassignedNeighbors(Node node)
    {
        List<Node> unAssigned = new ArrayList<Node>();
        for (Node neighbor : node.getNeighbors())
            if (neighbor.isAssigned() == -1)
                unAssigned.add(neighbor);
        return unAssigned;
    }


    /**
     * Returns a list of the Nodes that are included in the Dominating Set.
     * This implementation uses a Greedy Algorithm to find the dominating set 
     * of the graph. The Dominating Set, D, is a subset of nodes such that every
     * not in D is adjacent to at least one node in D.
     *
     * Note. This implementation on returns a Dominating Set. It is not guaranteed to
     * return the minimum dominating set.
     *
     * @param graph The graph object for which a dominating set is to be found.
     * 
     * @return A List of Nodes representing the nodes included in the dominating set.
     */
    public static List<Node> findGreedyDominatingSet(Graph graph)
    {
    	// keep track of number of nodes and number of uncovered nodes
    	boolean allNodesAreCovered = false;
        Node maxCoverNode;

        // keep track of nodes that have been visited
        List<Node> visited = new ArrayList<Node>();

    	// make sure all nodes become covered
    	while (!allNodesAreCovered)
    	{	
            allNodesAreCovered = false;

            // find the node that will cover the greatest number of uncovered nodes
    		int maxCoverNum = 0;
    		maxCoverNode = null;
			
    		for (Node node : graph.getNodes())
    		{	
    			if (node.isAssigned() != 1 )
    			{
	    			int uncoveredNeighbors = getNumberUncoveredNeighbors(node);
					if (uncoveredNeighbors > maxCoverNum)
					{
						maxCoverNum = uncoveredNeighbors;
						maxCoverNode = node;
					}
				}
    		}

    		// after finding max cover node, cover it an all of its neighbors
    		updateCover(maxCoverNode);

    		// add newMax to list of visited nodes
    		visited.add(maxCoverNode);

            // check that all nodes are covered
            allNodesAreCovered = true;
            for (Node node : graph.getNodes())
            {
                if (node.isAssigned() == -1)
                {
                    allNodesAreCovered = false;
                    break;
                }
            }
    	}
        graph.resetGraph();
    	return visited;
    }


    /**
     * Returns the number of neighbors of a given node that are uncovered.
     * @param name An integer representing the name of a node
     * @return An integer representing the number of uncovered neighbors
     */
    private static int getNumberUncoveredNeighbors(Node candidate)
    {
		int numUncovered = 0;

		if (candidate.isAssigned() != 1) { numUncovered++; }

		for (Node neighbor : candidate.getNeighbors() )
			if ( neighbor.isAssigned() == -1 ) {	numUncovered++; }

		return numUncovered;
    }


    /**
     * Updates the neighborhood of a node that has been assigned to 
     * the dominating set by marking all of its neighbors as out of the
     * dominating set.
     *
     * @param node The Node whose neighborhood needs to be updated.
     */
    private static void updateCover(Node node)
    {   
    	node.setAssignment(1);
        
        for (Node neighbor : node.getNeighbors() )
            if ( neighbor.isAssigned() == -1 ) 
                neighbor.setAssignment(0);
    }


    public static void main(String[] args)
    {   
        String filename = args[0];

        Graph g = new Graph();
        GraphLoader.loadGraph(g, filename);
        System.out.println("\nAnalyzing " + filename + " ...");
        System.out.println("Number of Nodes: " + g.getNumNodes());
        System.out.println("Number of Edges: " + g.getNumEdges());
        
        System.out.println("\nGreedy Dominating Set ...");
        List<Node> gds = findGreedyDominatingSet(g);
        System.out.println("Size of gds: " + gds.size() );
        System.out.println("Greedy dominating set: " + gds);

        System.out.println("\nMinimum Dominating Set ...");
        List<Node> mds = findMinimumDominatingSet(g);
        System.out.println("Size of mds with tree: " + mds.size() );
        System.out.println("Minimum Dominating Set: " + mds);
    }

}